In [3]:
import qval as qv

In [ ]:
A = qv.quantize(0.499999, 3)
A

In [ ]:
qv.dequantize(A)

In [ ]:
a = qv.qval('010')
b = qv.qval('110')

a.__mul__(b)

In [ ]:
q = qv.qval('001')

qv.qSigmoid(q)

In [ ]:
n1 = qv.quantize(0.5, 8)
n2 = qv.quantize(0.5, 4)


print(qv.dequantize(qv.qAdd(n1, n2)))
print(qv.dequantize(qv.qMul(n1, n2)))

In [ ]:
import torch

a = torch.tensor(0.5)
b = torch.tensor(0.5)

qa = qv.quantize(a.item(), 8)
qb = qv.quantize(b.item(), 8)


print(qv.dequantize(qv.qAdd(qa, qb)))
print(qv.dequantize(qv.qMul(qa, qb)))

In [ ]:
in_len = 10
out_len = 1
layer = torch.nn.Linear(in_len, out_len)
input_test = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

print(layer(input_test).item())

In [ ]:
# Quantize layer
from qval import qval

in_params, out_params = layer.parameters()
in_params = list(in_params)

if in_len > 1:
    in_params = in_params[0]

q_in_params = []
for param in in_params:
    # q_in_params.append(qv.quantize(param.item(), 8))
    q_in_params.append(qval(param.item(), 8))

if out_len > 1:
    out_params = out_params[0]

q_out_params = []
for param in out_params:
    # q_out_params.append(qv.quantize(param.item(), 8))
    q_out_params.append(qval(param.item(), 8))

print([qv.dequantize(x) for x in q_in_params])
print([param.item() for param in in_params])
print()
print()
print(qv.dequantize(q_out_params[0]), out_params[0].item())

In [ ]:
# print(q_in_params)
# print(input_test[0])
q_input = [qval(q.item(), 8) for q in input_test]

print([v.val() for v in q_input])

# result = qv.quantize(0.0, 8)
# for param, x in zip(q_in_params, q_input):
#     result = qv.qAdd(result, param * x)
    
# print(qv.dequantize(result))

In [13]:
def execute_layer(q_in_params, q_out_params, input_test, bits=8):
    q_input = [qval(q.item(), bits) for q in input_test]

    result = qval(0.0, bits)
    for param, x in zip(q_in_params, q_input):
        result = (result + param * x)

    return (result + q_out_params[0])

In [ ]:
execute_layer(q_in_params, q_out_params, input_test).val()

In [ ]:
import random
random_values = [random.random() for _ in range(10)]
random_input = torch.tensor(random_values)

original = layer(random_input).item()
quantized = execute_layer(q_in_params, q_out_params, random_input).val()

print(original, quantized)
print(f"Diff: {abs(original - quantized)}")

if abs(original - quantized) > 0.3:
    print([x.item() for x in random_input])
    q_input = [qval(x, 8) for x in random_values]

    print([q.val for q in q_input])

In [ ]:
for _ in range(10000):
    random_values = [random.random() for _ in range(10)]
    random_input = torch.tensor(random_values)

    original = layer(random_input).item()
    quantized = execute_layer(q_in_params, q_out_params, random_input).val()

    # print(original, quantized)
    # print(f"Diff: {abs(original - quantized)}")
    # print(abs(original - quantized))

    if abs(original - quantized) > 0.3:
        print([x.item() for x in random_input])
        q_input = [qval(x, 8) for x in random_values]

        print([q.val for q in q_input])

In [ ]:
from qval.qlayer import qdense

input_params = [[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] for _ in range(10)]
out_params = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
teste = qdense(input_params, out_params)
teste

In [ ]:
input_test = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 5.0]

output = teste.process(input_test)

[x.val() for x in output]

In [ ]:
qval(6.5)

In [ ]:
qval(6.5, scale=100)